# load packages

In [1]:
from scipy.sparse import coo_matrix
import numpy as np
import scipy as sp
import pandas as pd

To set up access token, see [here](https://connectome-neuprint.github.io/neuprint-python/docs/quickstart.html).

In [2]:
from neuprint import Client, fetch_adjacencies,fetch_neurons, NeuronCriteria as NC

c = Client('neuprint.janelia.org', dataset='optic-lobe:v1.1')

# getting data

In [3]:
# connectivity between all neurons 

# The cell types defined as optic neuropil according to Nern2024 (DOI:10.1101/2024.04.16.589741) are listed
#   below. Alternatively import the code from 
#   https://github.com/reiserlab/male-drosophila-visual-system-connectome-code and run:
#   ```{.python}
#     olt = OLTypes()
#     on_list = olt.get_neuron_list()
#     on_types = on_list['type']
#   ```
on_types = [ '5-HTPMPV01', '5-HTPMPV03', '5thsLNv_LNd6', 'Am1', 'aMe1', 'aMe2', 'aMe3', 'aMe4', 'aMe5', 
            'aMe6a', 'aMe6b', 'aMe6c', 'aMe8', 'aMe9', 'aMe10', 'aMe12', 'aMe13', 'aMe15', 'aMe17a', 
            'aMe17b', 'aMe17c', 'aMe17e', 'aMe20', 'aMe22', 'aMe23', 'aMe24', 'aMe25', 'aMe26', 'aMe30', 
            'aMe_TBD1', 'AN09A005', 'AN27X013', 'AOTU045', 'AOTU056', 'AOTU058', 'C2', 'C3', 'CL125', 
            'CL357', 'Cm-DRA', 'Cm1', 'Cm2', 'Cm3', 'Cm4', 'Cm5', 'Cm6', 'Cm7', 'Cm8', 'Cm9', 'Cm10', 
            'Cm11a', 'Cm11b', 'Cm11c', 'Cm11d', 'Cm12', 'Cm13', 'Cm14', 'Cm15', 'Cm16', 'Cm17', 'Cm18', 
            'Cm19', 'Cm20', 'Cm21', 'Cm22', 'Cm23', 'Cm24', 'Cm25', 'Cm26', 'Cm27', 'Cm28', 'Cm29', 'Cm30', 
            'Cm31a', 'Cm31b', 'Cm32', 'Cm33', 'Cm34', 'Cm35', 'CT1', 'dCal1', 'DCH', 'Dm-DRA1', 'Dm-DRA2', 
            'Dm1', 'Dm2', 'Dm3a', 'Dm3b', 'Dm3c', 'Dm4', 'Dm6', 'Dm8a', 'Dm8b', 'Dm9', 'Dm10', 'Dm11', 'Dm12', 
            'Dm13', 'Dm14', 'Dm15', 'Dm16', 'Dm17', 'Dm18', 'Dm19', 'Dm20', 'DN1a', 'DNc01', 'DNc02', 'DNp11', 
            'DNp27', 'DNp30', 'DNpe053', 'H1', 'H2', 'HBeyelet', 'HSE', 'HSN', 'HSS', 'HST', 'KCg-s1', 'l-LNv', 
            'L1', 'L2', 'L3', 'L4', 'L5', 'Lai', 'LAL047', 'LAL048', 'Lat1', 'Lat2', 'Lat3', 'Lat4', 'Lat5', 
            'Lawf1', 'Lawf2', 'LC4', 'LC6', 'LC9', 'LC10a', 'LC10b', 'LC10c-1', 'LC10c-2', 'LC10d', 'LC10e', 
            'LC11', 'LC12', 'LC13', 'LC14a-1', 'LC14a-2', 'LC14b', 'LC15', 'LC16', 'LC17', 'LC18', 'LC19', 
            'LC20a', 'LC20b', 'LC21', 'LC22', 'LC23', 'LC24', 'LC25', 'LC26', 'LC27', 'LC28', 'LC29', 'LC30', 
            'LC31a', 'LC31b', 'LC33', 'LC34', 'LC35a', 'LC35b', 'LC36', 'LC37', 'LC39a', 'LC39b', 'LC40', 
            'LC41', 'LC43', 'LC44', 'LC46b', 'Li11a', 'Li11b', 'Li12', 'Li13', 'Li14', 'Li15', 'Li16', 
            'Li17', 'Li18a', 'Li18b', 'Li19', 'Li20', 'Li21', 'Li22', 'Li23', 'Li25', 'Li26', 'Li27', 
            'Li28', 'Li29', 'Li30', 'Li31', 'Li32', 'Li33', 'Li34a', 'Li34b', 'Li35', 'Li36', 'Li37', 
            'Li38', 'Li39', 'LLPC1', 'LLPC2', 'LLPC3', 'LLPC4', 'LOLP1', 'LoVC1', 'LoVC2', 'LoVC3', 
            'LoVC4', 'LoVC5', 'LoVC6', 'LoVC7', 'LoVC9', 'LoVC11', 'LoVC12', 'LoVC13', 'LoVC14', 
            'LoVC15', 'LoVC16', 'LoVC17', 'LoVC18', 'LoVC19', 'LoVC20', 'LoVC21', 'LoVC22', 'LoVC23', 
            'LoVC24', 'LoVC25', 'LoVC26', 'LoVC27', 'LoVC28', 'LoVC29', 'LoVCLo1', 'LoVCLo2', 'LoVCLo3', 
            'LoVP1', 'LoVP2', 'LoVP3', 'LoVP4', 'LoVP5', 'LoVP6', 'LoVP7', 'LoVP8', 'LoVP9', 'LoVP10', 
            'LoVP11', 'LoVP12', 'LoVP13', 'LoVP14', 'LoVP16', 'LoVP17', 'LoVP18', 'LoVP19', 'LoVP20', 
            'LoVP21', 'LoVP22', 'LoVP23', 'LoVP24', 'LoVP25', 'LoVP26', 'LoVP27', 'LoVP28', 'LoVP29', 
            'LoVP30', 'LoVP31', 'LoVP32', 'LoVP33', 'LoVP34', 'LoVP35', 'LoVP36', 'LoVP37', 'LoVP38', 
            'LoVP39', 'LoVP40', 'LoVP41', 'LoVP42', 'LoVP43', 'LoVP44', 'LoVP45', 'LoVP46', 'LoVP47', 
            'LoVP48', 'LoVP49', 'LoVP50', 'LoVP51', 'LoVP52', 'LoVP53', 'LoVP54', 'LoVP55', 'LoVP56', 
            'LoVP57', 'LoVP58', 'LoVP59', 'LoVP60', 'LoVP61', 'LoVP62', 'LoVP63', 'LoVP64', 'LoVP65', 
            'LoVP66', 'LoVP67', 'LoVP68', 'LoVP69', 'LoVP70', 'LoVP71', 'LoVP72', 'LoVP73', 'LoVP74', 
            'LoVP75', 'LoVP76', 'LoVP77', 'LoVP78', 'LoVP79', 'LoVP80', 'LoVP81', 'LoVP82', 'LoVP83', 
            'LoVP84', 'LoVP85', 'LoVP86', 'LoVP88', 'LoVP89', 'LoVP90a', 'LoVP90b', 'LoVP90c', 'LoVP91', 
            'LoVP92', 'LoVP93', 'LoVP94', 'LoVP95', 'LoVP96', 'LoVP97', 'LoVP98', 'LoVP99', 'LoVP100', 
            'LoVP101', 'LoVP102', 'LoVP103', 'LoVP105', 'LoVP106', 'LoVP107', 'LoVP108', 'LPC1', 'LPC2', 
            'LPi2b', 'LPi2c', 'LPi2d', 'LPi2e', 'LPi3a', 'LPi3b', 'LPi3c', 'LPi4a', 'LPi4b', 'LPi12', 
            'LPi14', 'LPi21', 'LPi34', 'LPi43', 'LPi3412', 'LPLC1', 'LPLC2', 'LPLC4', 'LPT21', 'LPT22', 
            'LPT23', 'LPT26', 'LPT27', 'LPT28', 'LPT29', 'LPT30', 'LPT31', 'LPT49', 'LPT50', 'LPT51', 
            'LPT52', 'LPT53', 'LPT54', 'LPT57', 'LPT59', 'LPT60', 'LPT100', 'LPT101', 'LPT110', 'LPT111', 
            'LPT112', 'LPT113', 'LPT114', 'LPT115', 'LPT116', 'LT1a', 'LT1b', 'LT1c', 'LT1d', 'LT11', 
            'LT33', 'LT34', 'LT35', 'LT36', 'LT37', 'LT39', 'LT40', 'LT41', 'LT42', 'LT43', 'LT46', 
            'LT47', 'LT51', 'LT52', 'LT54', 'LT55', 'LT56', 'LT58', 'LT59', 'LT60', 'LT61a', 'LT61b', 
            'LT62', 'LT63', 'LT64', 'LT65', 'LT66', 'LT67', 'LT68', 'LT69', 'LT70', 'LT72', 'LT73', 
            'LT74', 'LT75', 'LT76', 'LT77', 'LT78', 'LT79', 'LT80', 'LT81', 'LT82a', 'LT82b', 'LT83', 
            'LT84', 'LT85', 'LT86', 'LT87', 'LT88', 'mALD1', 'MeLo1', 'MeLo2', 'MeLo3a', 'MeLo3b', 
            'MeLo4', 'MeLo5', 'MeLo6', 'MeLo7', 'MeLo8', 'MeLo9', 'MeLo10', 'MeLo11', 'MeLo12', 'MeLo13', 
            'MeLo14', 'MeTu1', 'MeTu2a', 'MeTu2b', 'MeTu3a', 'MeTu3b', 'MeTu3c', 'MeTu4a', 'MeTu4b', 
            'MeTu4c', 'MeTu4d', 'MeTu4e', 'MeTu4f', 'MeVC1', 'MeVC2', 'MeVC3', 'MeVC4a', 'MeVC4b', 
            'MeVC5', 'MeVC6', 'MeVC7a', 'MeVC7b', 'MeVC8', 'MeVC9', 'MeVC10', 'MeVC11', 'MeVC12', 
            'MeVC20', 'MeVC21', 'MeVC22', 'MeVC23', 'MeVC24', 'MeVC25', 'MeVC26', 'MeVC27', 'MeVCMe1', 
            'MeVP1', 'MeVP2', 'MeVP3', 'MeVP4', 'MeVP5', 'MeVP6', 'MeVP7', 'MeVP8', 'MeVP9', 'MeVP10', 
            'MeVP11', 'MeVP12', 'MeVP14', 'MeVP15', 'MeVP16', 'MeVP17', 'MeVP18', 'MeVP20', 'MeVP21', 
            'MeVP22', 'MeVP23', 'MeVP24', 'MeVP25', 'MeVP26', 'MeVP27', 'MeVP28', 'MeVP29', 'MeVP30', 
            'MeVP31', 'MeVP32', 'MeVP33', 'MeVP34', 'MeVP35', 'MeVP36', 'MeVP38', 'MeVP39', 'MeVP40', 
            'MeVP41', 'MeVP42', 'MeVP43', 'MeVP45', 'MeVP46', 'MeVP47', 'MeVP48', 'MeVP49', 'MeVP50', 
            'MeVP51', 'MeVP52', 'MeVP53', 'MeVP54', 'MeVP55', 'MeVP56', 'MeVP57', 'MeVP58', 'MeVP59', 
            'MeVP60', 'MeVP61', 'MeVP62', 'MeVP63', 'MeVP64', 'MeVPaMe1', 'MeVPaMe2', 'MeVPLo1', 
            'MeVPLo2', 'MeVPLp1', 'MeVPLp2', 'MeVPMe1', 'MeVPMe2', 'MeVPMe3', 'MeVPMe4', 'MeVPMe5', 
            'MeVPMe6', 'MeVPMe7', 'MeVPMe8', 'MeVPMe9', 'MeVPMe10', 'MeVPMe11', 'MeVPMe12', 'MeVPMe13', 
            'MeVPOL1', 'Mi1', 'Mi2', 'Mi4', 'Mi9', 'Mi10', 'Mi13', 'Mi14', 'Mi15', 'Mi16', 'Mi17', 'Mi18', 
            'Mi19', 'Mi20', 'Nod1', 'Nod2', 'Nod3', 'Nod4', 'Nod5', 'OA-AL2i1', 'OA-AL2i2', 'OA-AL2i3', 
            'OA-AL2i4', 'OA-ASM1', 'OLVC1', 'OLVC2', 'OLVC3', 'OLVC4', 'OLVC5', 'OLVC6', 'OLVC7', 'PLP021', 
            'PLP032', 'PLP036', 'PLP069', 'PLP080', 'PLP150', 'PLP211', 'PLP231', 'PLP_TBD1', 'Pm1', 'Pm2a', 
            'Pm2b', 'Pm3', 'Pm4', 'Pm5', 'Pm6', 'Pm7', 'Pm8', 'Pm9', 'Pm10', 'Pm11', 'Pm12', 'Pm13', 'PS272', 
            'PVLP046', 'R1-R6', 'R7d', 'R7p', 'R7y', 'R8d', 'R8p', 'R8y', 's-LNv', 'SLP249', 'SLP250', 'SLP359', 
            'SMP200', 'SMP217', 'SMP528', 'T1', 'T2', 'T2a', 'T3', 'T4a', 'T4b', 'T4c', 'T4d', 'T5a', 'T5b', 
            'T5c', 'T5d', 'Tlp11', 'Tlp12', 'Tlp13', 'Tlp14', 'Tm1', 'Tm2', 'Tm3', 'Tm4', 'Tm5a', 'Tm5b', 
            'Tm5c', 'Tm5Y', 'Tm6', 'Tm9', 'Tm12', 'Tm16', 'Tm20', 'Tm23', 'Tm24', 'Tm26', 'Tm29', 'Tm30', 
            'Tm31', 'Tm32', 'Tm33', 'Tm34', 'Tm35', 'Tm36', 'Tm37', 'Tm38', 'Tm39', 'Tm40', 'TmY3', 'TmY4', 
            'TmY5a', 'TmY9a', 'TmY9b', 'TmY10', 'TmY13', 'TmY14', 'TmY15', 'TmY16', 'TmY17', 'TmY18', 
            'TmY19a', 'TmY19b', 'TmY20', 'TmY21', 'V1', 'vCal1', 'vCal2', 'vCal3', 'VCH', 'VLP_TBD1', 
            'VS', 'VSm', 'VST1', 'VST2', 'Y3', 'Y11', 'Y12', 'Y13', 'Y14']

neuron_df, conn_df = fetch_adjacencies(NC(type=on_types), NC(type=on_types))

  0%|          | 0/265 [00:00<?, ?it/s]

# inspect

In [4]:
neuron_df

,bodyId,type,instance
0,10009,CT1,CT1_L
1,10012,VS,VS_R
2,10015,HSN,HSN_R
3,10016,HSE,HSE_R
4,10023,HSS,HSS_R
...,...,...,...
52822,867122111,R1-R6,R1-R6_R
52823,876890618,R1-R6,R1-R6_R
52824,966201979,R1-R6,R1-R6_R
52825,967550825,R7d,R7d_R


In [6]:
# is 'instance' column just 'type' + ending on '_L/R'? 
neuron_df.loc[:,'instance_noside'] = neuron_df['instance'].str.replace('_[LR]$','', regex=True)
(neuron_df.type == neuron_df.instance_noside).value_counts()

True    52827
Name: count, dtype: int64

In [7]:
# get ones that are not the same
neuron_df[neuron_df.type != neuron_df.instance_noside]

,bodyId,type,instance,instance_noside


In [8]:
# seems like we can just use the type column 
# and use the instance for side 
neuron_df.loc[:,'side'] = neuron_df['instance'].str[-1]
# replace 'L' with 'left' and 'R' with 'right'
neuron_df.loc[:,'side'] = neuron_df['side'].replace({'L':'left','R':'right'})
neuron_df

,bodyId,type,instance,instance_noside,side
0,10009,CT1,CT1_L,CT1,left
1,10012,VS,VS_R,VS,right
2,10015,HSN,HSN_R,HSN,right
3,10016,HSE,HSE_R,HSE,right
4,10023,HSS,HSS_R,HSS,right
...,...,...,...,...,...
52822,867122111,R1-R6,R1-R6_R,R1-R6,right
52823,876890618,R1-R6,R1-R6_R,R1-R6,right
52824,966201979,R1-R6,R1-R6_R,R1-R6,right
52825,967550825,R7d,R7d_R,R7d,right


In [9]:
neuron_df.type.value_counts()

type
R1-R6     2258
Tm3       1037
T3         976
T2a        939
L5         898
          ... 
LoVP28       1
LAL047       1
LT33         1
LT56         1
LoVP91       1
Name: count, Length: 732, dtype: int64

In [10]:
neuron_df[neuron_df.type.str.contains('unclear')].type.value_counts()

Series([], Name: count, dtype: int64)

In [11]:
# replace the '_unclear' with ''
neuron_df.type = neuron_df.type.str.replace('_unclear', '')

In [12]:
conn_df

,bodyId_pre,bodyId_post,roi,weight
0,10009,10015,LO(R),7
1,10009,10015,NotPrimary,2
2,10009,10029,NotPrimary,3
3,10009,10029,LO(R),1
4,10009,10046,ME(R),1
...,...,...,...,...
6718926,983968288,105609,ME(R),2
6718927,983968288,143501,ME(R),1
6718928,983968288,178350,ME(R),1
6718929,983968288,250082,ME(R),3


In [13]:
conn_df.roi.unique()

array(['LO(R)', 'NotPrimary', 'ME(R)', 'LOP(R)', 'PLP(R)', 'AME(R)',
       'LA(R)', 'PVLP(R)'], dtype=object)

In [14]:
conn = conn_df.groupby(['bodyId_pre', 'bodyId_post']).weight.sum().reset_index()
conn

,bodyId_pre,bodyId_post,weight
0,10009,10015,9
1,10009,10029,4
2,10009,10046,1
3,10009,10053,7
4,10009,10072,14
...,...,...,...
6469437,983968288,105609,2
6469438,983968288,143501,1
6469439,983968288,178350,1
6469440,983968288,250082,3


In [15]:
# instead of making a dense matrix based on the edgelist above, let's make a sparse one from the edgelist directly
# first make a coo matrix
nodes = set(neuron_df.bodyId)
sorted_nodes = sorted(nodes)  # Convert the set to a sorted list
nodes_to_idx = {node: num for num, node in enumerate(sorted_nodes)}

# type to type connectivity
conn['pre_idx'] = conn.bodyId_pre.map(nodes_to_idx)
conn['post_idx'] = conn.bodyId_post.map(nodes_to_idx)

# Create COO matrix
row = conn['pre_idx'].values
col = conn['post_idx'].values
data = conn['weight'].values
matrix_size = len(nodes)
coo = coo_matrix((data, (row, col)), shape=(matrix_size, matrix_size))

# then turn it into csc matrix
csc = coo.tocsc()

# calculate the size
csc_size = csc.data.nbytes  # Size of the data array
csc_size += csc.indices.nbytes  # Size of the indices array
csc_size += csc.indptr.nbytes  # Size of the index pointer array
# number of MB
csc_size/1e6

77.844616

In [16]:
csc.shape

(52827, 52827)

In [17]:
col_sums = csc.sum(axis=0)
# Handling division by zero in case some columns have a sum of zero
# that is, where a neuron doesn't have incoming synapses
# .A turns it from a sparse matrix into a dense np array
col_sums_with_inversion = np.reciprocal(
    col_sums.A.squeeze().astype(float), where=col_sums.A.squeeze() != 0)
# Multiply each column by the inverse of its sum
inprop = csc.multiply(col_sums_with_inversion)
# and then reduce the precision to float32 to save memory
inprop = inprop.astype(np.float32)

In [18]:
sp.sparse.save_npz(
    'data/neuprint_inprop_optic.npz', inprop)

In [19]:
neuron_df

,bodyId,type,instance,instance_noside,side
0,10009,CT1,CT1_L,CT1,left
1,10012,VS,VS_R,VS,right
2,10015,HSN,HSN_R,HSN,right
3,10016,HSE,HSE_R,HSE,right
4,10023,HSS,HSS_R,HSS,right
...,...,...,...,...,...
52822,867122111,R1-R6,R1-R6_R,R1-R6,right
52823,876890618,R1-R6,R1-R6_R,R1-R6,right
52824,966201979,R1-R6,R1-R6_R,R1-R6,right
52825,967550825,R7d,R7d_R,R7d,right


# NT

In [20]:
meta, _ = fetch_neurons(neuron_df.bodyId)

In [21]:
meta

,bodyId,instance,type,pre,post,downstream,upstream,size,status,statusLabel,...,entryNerve,mancGroup,flywireId,flywireType,mcnsSerial,celltypeTotalNtPredictions,predictedNtConfidence,totalNtPredictions,inputRois,outputRois
0,10009,CT1_L,CT1,31713,82800,189641,82800,30164249462,Traced,Roughly traced,...,None,NaN,None,CT1,None,31535,0.776037,31535,"[LAL(R), LO(R), LOP(R), LOP_R_col_02_09, LOP_R...","[LAL(R), LO(R), LOP(R), LOP_R_col_05_19, LOP_R..."
1,10012,VS_R,VS,923,18867,4817,18867,18184684149,Traced,Roughly traced,...,None,NaN,None,None,None,2756,0.334622,437,"[GNG, IPS(R), LOP(R), LOP_R_col_08_02, LOP_R_c...","[GNG, IPS(R), LOP(R), LOP_R_col_12_03, LOP_R_c..."
2,10015,HSN_R,HSN,913,17265,4903,17265,19592174183,Traced,Roughly traced,...,None,NaN,None,HSN,None,487,0.456840,487,"[GNG, IPS(R), LO(R), LOP(R), LOP_R_col_16_20, ...","[GNG, IPS(R), LO(R), LOP(R), LOP_R_col_17_32, ..."
3,10016,HSE_R,HSE,515,18816,2645,18816,19390346866,Traced,Roughly traced,...,None,NaN,None,HSE,None,35,0.366792,35,"[GNG, IPS(R), LO(R), LOP(R), LOP_R_col_07_18, ...","[GNG, IPS(R), LOP(R), LOP_R_col_14_12, LOP_R_c..."
4,10023,HSS_R,HSS,886,18241,5382,18241,19387850164,Traced,Roughly traced,...,None,NaN,None,HSS,None,205,0.468701,205,"[GNG, IPS(R), LOP(R), LOP_R_col_03_08, LOP_R_c...","[GNG, IPS(R), LOP(R), LOP_R_col_05_12, LOP_R_c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52822,867122111,R1-R6_R,R1-R6,1,1,4,1,3520376,Anchor,Primary Anchor,...,None,NaN,None,R1-6,None,74921,0.000000,0,"[LA(R), OL(R)]","[LA(R), OL(R)]"
52823,876890618,R1-R6_R,R1-R6,1,0,3,0,65168,Anchor,Primary Anchor,...,None,NaN,None,R1-6,None,74921,0.000000,0,[],"[LA(R), OL(R)]"
52824,966201979,R1-R6_R,R1-R6,21,14,91,14,48883331,Orphan,Orphan hotknife,...,None,NaN,None,R1-6,None,74921,0.960561,21,"[LA(R), OL(R)]","[LA(R), OL(R)]"
52825,967550825,R7d_R,R7d,40,89,306,89,86894628,Traced,Prelim Roughly traced,...,None,NaN,None,None,None,2555,0.865494,40,"[ME(R), ME_R_col_30_18, ME_R_col_31_17, ME_R_c...","[ME(R), ME_R_col_31_17, ME_R_col_32_18, ME_R_l..."


In [22]:
meta.columns

Index(['bodyId', 'instance', 'type', 'pre', 'post', 'downstream', 'upstream',
       'size', 'status', 'statusLabel', 'somaLocation', 'roiInfo',
       'celltypePredictedNtConfidence', 'assignedOlHex2', 'serialMotif',
       'trumanHl', 'matchingNotes', 'synweight', 'hemilineage',
       'otherNtReference', 'locationType', 'assignedOlHex1', 'ntReference',
       'otherNt', 'birthtime', 'exitNerve', 'celltypePredictedNt',
       'somaNeuromere', 'itoleeHl', 'dimorphism', 'predictedNt', 'consensusNt',
       'entryNerve', 'mancGroup', 'flywireId', 'flywireType', 'mcnsSerial',
       'celltypeTotalNtPredictions', 'predictedNtConfidence',
       'totalNtPredictions', 'inputRois', 'outputRois'],
      dtype='object')

In [23]:
meta[['type','consensusNt', 'totalNtPredictions',
      'celltypePredictedNt', 'otherNtReference', 'predictedNt', 'otherNt', 'ntReference', 'predictedNtConfidence', 'celltypePredictedNtConfidence', 'celltypeTotalNtPredictions']]

,type,consensusNt,totalNtPredictions,celltypePredictedNt,otherNtReference,predictedNt,otherNt,ntReference,predictedNtConfidence,celltypePredictedNtConfidence,celltypeTotalNtPredictions
0,CT1,gaba,31535,gaba,None,gaba,None,Takemura et al 2017,0.776037,0.776037,31535
1,VS,acetylcholine,437,dopamine,None,unclear,None,Zhao et al 2023,0.334622,0.394740,2756
2,HSN,acetylcholine,487,histamine,None,unclear,None,Zhao et al 2023,0.456840,0.456840,487
3,HSE,acetylcholine,35,unclear,None,unclear,None,Zhao et al 2023,0.366792,0.366792,35
4,HSS,acetylcholine,205,histamine,None,unclear,None,Zhao et al 2023,0.468701,0.468701,205
...,...,...,...,...,...,...,...,...,...,...,...
52822,R1-R6,histamine,0,histamine,None,unclear,None,Davis et al 2020,0.000000,0.949296,74921
52823,R1-R6,histamine,0,histamine,None,unclear,None,Davis et al 2020,0.000000,0.949296,74921
52824,R1-R6,histamine,21,histamine,None,unclear,None,Davis et al 2020,0.960561,0.949296,74921
52825,R7d,histamine,40,histamine,None,unclear,None,None,0.865494,0.730862,2555


In [28]:
meta[['consensusNt','celltypePredictedNt']].value_counts(dropna=False)

consensusNt    celltypePredictedNt
acetylcholine  acetylcholine          32279
glutamate      glutamate              10283
gaba           gaba                    5424
histamine      histamine               3648
unclear        glutamate                892
               dopamine                 563
               unclear                  556
               serotonin                 28
               octopamine                21
octopamine     octopamine                12
acetylcholine  unclear                   10
               dopamine                   9
serotonin      serotonin                  7
glutamate      octopamine                 5
dopamine       dopamine                   4
gaba           dopamine                   2
acetylcholine  octopamine                 2
               histamine                  2
Name: count, dtype: int64

In [24]:
# use `consensusNt`: Nern2024 provides this "best guess" based on `celltypePredictedNt` (Eckstein2024) 
# and a range of experiments. `celltypePredictedNt` had a high reliability of 92.5% across 543 cell types that were
# tested. The `consensusNt` gives more realistic results at this time.

bodyid_to_nt = dict(zip(meta.bodyId, meta.consensusNt))
# for now regarding everything except for Glu and GABA to be excitatory
# based on papers such as this (https://pubmed.ncbi.nlm.nih.gov/7472349/), when light hits photoreceptors, they reduce the release of histamine. 
# histamine inhibits the large monopolar cells 
# so excitation of photoreceptors excites the large monopolar cells (disinhibition). 
# so we'll model this as excitation, i.e. 'histamine': 1 
nt_to_sign = dict.fromkeys(meta.consensusNt.unique(), 1)
nt_to_sign.update({'gaba': -1, 'glutamate': -1})

# save meta

In [25]:
meta = neuron_df[['bodyId', 'type', 'side']].copy()
meta.rename(columns={'type': 'cell_type'}, inplace=True)
meta['idx'] = meta['bodyId'].map(nodes_to_idx)
meta.loc[:,['nt']] = meta['bodyId'].map(bodyid_to_nt)
meta.loc[:,['sign']] = meta['nt'].map(nt_to_sign)

meta.to_csv('data/neuprint_meta_optic.csv', index=False)
meta

,bodyId,cell_type,side,idx,nt,sign
0,10009,CT1,left,0,gaba,-1
1,10012,VS,right,1,acetylcholine,1
2,10015,HSN,right,2,acetylcholine,1
3,10016,HSE,right,3,acetylcholine,1
4,10023,HSS,right,4,acetylcholine,1
...,...,...,...,...,...,...
52822,867122111,R1-R6,right,52822,histamine,1
52823,876890618,R1-R6,right,52823,histamine,1
52824,966201979,R1-R6,right,52824,histamine,1
52825,967550825,R7d,right,52825,histamine,1
